In [5]:
import pandas as pd
from dash import Dash, dcc, html, Input, Output
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Load data
file_path = r"C:\Users\Raabiyaal Ishaq\OneDrive - The University of Chicago\Desktop\2. RA-SHIP\SUMMER PROJECTS\ACLI\Python\dash-app\All Prop Type.xlsx"
df = pd.read_excel(file_path, header=1)

# Load recession data
recession_file_path = r"C:\Users\Raabiyaal Ishaq\OneDrive - The University of Chicago\Desktop\2. RA-SHIP\SUMMER PROJECTS\ACLI\Python\dash-app\Recession Data.xlsx"  # update path

recession_df = pd.read_excel(recession_file_path, header=1)
recession_df['observation_date'] = pd.to_datetime(recession_df['observation_date'])





rf_file_path = r"C:\Users\Raabiyaal Ishaq\OneDrive - The University of Chicago\Desktop\2. RA-SHIP\SUMMER PROJECTS\ACLI\Python\dash-app\Rf.xlsx" # Update this

rf_df = pd.read_excel(rf_file_path, header=1)  # Header is in second row (index 1)
rf_df['Quarter'] = pd.to_datetime(rf_df['Quarter'])

rf_df['Rf'] = rf_df['Rf'].astype(str).str.replace('%', '').astype(float)


# Clean data
if 'Unnamed: 0' in df.columns:
    df = df.drop(columns=['Unnamed: 0'])

df['Quarter'] = pd.to_datetime(df['Quarter'])

# Keep as percent values (no division by 100)
for col in ['Rf', 'g', 'LTV: 25%', 'LTV: 50%', 'LTV: 75%']:
    df[col] = df[col].astype(str).str.replace('%', '').astype(float)

df['Loan Committed'] = df['Loan Committed'].astype(str).str.replace(',', '').astype(float).astype(int)

# Melt LTV columns for plotting
melted = df.melt(id_vars=['Property Type', 'Quarter', 'Loan Committed'],
                 value_vars=['LTV: 25%', 'LTV: 50%', 'LTV: 75%'],
                 var_name='LTV', value_name='Interest Rate')

# Base RGB colors for each property type
base_colors = {
    'Industrial': '255, 165, 0',
    'Office': '70, 130, 180',
    'Retail': '255, 105, 180',
    'Apartment': '60, 179, 113',
    'Core': '255, 215, 0'
}

# Fill colors with opacity
fill_colors = {
    'Apartment': {
        'LTV: 25%': 'rgba(60,179,113,0.5)',
        'LTV: 50%': 'rgba(60,179,113,0.3)',
        'LTV: 75%': 'rgba(60,179,113,0.15)'
    },
    'Core': {
        'LTV: 25%': 'rgba(255,215,0,0.5)',
        'LTV: 50%': 'rgba(255,215,0,0.3)',
        'LTV: 75%': 'rgba(255,215,0,0.15)'
    },
    'Industrial': {
        'LTV: 25%': 'rgba(255,165,0,0.5)',
        'LTV: 50%': 'rgba(255,165,0,0.3)',
        'LTV: 75%': 'rgba(255,165,0,0.15)'
    },
    'Office': {
        'LTV: 25%': 'rgba(70,130,180,0.5)',
        'LTV: 50%': 'rgba(70,130,180,0.3)',
        'LTV: 75%': 'rgba(70,130,180,0.15)'
    },
    'Retail': {
        'LTV: 25%': 'rgba(255,105,180,0.5)',
        'LTV: 50%': 'rgba(255,105,180,0.3)',
        'LTV: 75%': 'rgba(255,105,180,0.15)'
    }
}

def get_line_color(prop_type):
    rgb = base_colors.get(prop_type, '100,100,100')
    return f'rgba({rgb},1)'

def darken_color(rgb_str, amount=0.5):
    r, g, b = [int(x.strip()) for x in rgb_str.split(',')]
    r_d = max(0, int(r * amount))
    g_d = max(0, int(g * amount))
    b_d = max(0, int(b * amount))
    return f'rgba({r_d},{g_d},{b_d},1)'

def get_loan_color(prop_type):
    rgb = base_colors.get(prop_type, '100,100,100')
    return darken_color(rgb, amount=0.5)

app = Dash(__name__)

app.layout = html.Div([
    html.H2("Interactive Property Type & LTV Interest Rates"),

    html.Label("Select Property Type(s):"),
    dcc.Dropdown(
        id='property-type-dropdown',
        options=[{'label': pt, 'value': pt} for pt in df['Property Type'].unique()],
        value=['Apartment'],
        multi=True,
        clearable=False,
    ),

    html.Br(),

    html.Label("Select LTV Levels:"),
    dcc.Checklist(
        id='additional-lines-checklist',
        options=[
            {'label': 'Gamma (γ)', 'value': 'gamma'},
            {'label': 'Risk-Free Rate (Rf)', 'value': 'rf'},
            {'label': 'Loan Volume', 'value': 'loan'}
        ],
        value=['gamma', 'rf', 'loan'],  # default all three checked
        inline=True
    ),


    html.Br(),

    html.Label("Select Additional Lines:"),
    dcc.Checklist(
        id='additional-lines-checklist',
        options=[
            {'label': 'Gamma (γ)', 'value': 'gamma'},
            {'label': 'Risk-Free Rate (Rf)', 'value': 'rf'}
        ],
        value=['gamma', 'rf'],  # default both checked; adjust if needed
        inline=True
    ),

    dcc.Graph(
        id='interest-loan-graph',
        style={
            'border': '1px solid lightgray',
            'padding': '10px',
            'borderRadius': '8px'
        }
    )
])





@app.callback(
    Output('interest-loan-graph', 'figure'),
    Input('property-type-dropdown', 'value'),
    Input('ltv-checklist', 'value'),
    Input('additional-lines-checklist', 'value')  # NEW INPUT
)
def update_graph(selected_properties, selected_ltvs, additional_lines):
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    if not selected_properties:
        return go.Figure()

    for prop_type in selected_properties:
        # LTV lines with area fill
        for ltv in selected_ltvs:
            sub_df = melted[(melted['Property Type'] == prop_type) & (melted['LTV'] == ltv)]
            line_color = get_line_color(prop_type)
            fillcolor = fill_colors.get(prop_type, {}).get(ltv, 'rgba(200,200,200,0.2)')

            fig.add_trace(go.Scatter(
                x=sub_df['Quarter'],
                y=sub_df['Interest Rate'],
                mode='lines',
                fill='tonexty',
                name=f"{prop_type} - {ltv}",
                line=dict(color=line_color),
                fillcolor=fillcolor,
                hovertemplate='%{y:.2%} %{fullData.name}<extra></extra>'
            ), secondary_y=False)

        # Add Gamma if selected
        if 'gamma' in additional_lines:
            g_df = df[df['Property Type'] == prop_type]
            gamma_color = darken_color(base_colors.get(prop_type, '100,100,100'), amount=0.7)
            fig.add_trace(go.Scatter(
                x=g_df['Quarter'],
                y=g_df['g'],
                mode='lines',
                name=f"{prop_type} \u03B3",
                line=dict(color=gamma_color, dash='dash'),
                hovertemplate='%{y:.2%} %{fullData.name}<extra></extra>'
            ), secondary_y=False)

        # Add Loan Volume (always shown)
# Add Loan Volume if selected
    if 'loan' in additional_lines:
        loan_df = df[df['Property Type'] == prop_type]
        loan_color = get_loan_color(prop_type)
        fig.add_trace(go.Scatter(
            x=loan_df['Quarter'],
            y=[val / 1e9 for val in loan_df['Loan Committed']],
            mode='lines',
            name=f'{prop_type} Loan Volume',
            line=dict(color=loan_color, width=2),
            hovertemplate='$%{y:.2f}B<br>%{fullData.name}<extra></extra>'
        ), secondary_y=True)


    # Add Rf (risk-free rate) only once if selected
    if 'rf' in additional_lines:
        fig.add_trace(go.Scatter(
            x=rf_df['Quarter'],
            y=rf_df['Rf'],
            mode='lines',
            name='Risk-Free Rate (Rf)',
            line=dict(color='black', dash='dot'),
            hovertemplate='%{y:.2%} %{fullData.name}<extra></extra>'
        ), secondary_y=False)

    # Add thick vertical lines for each recession quarter
    recession_quarters = recession_df[recession_df['USREC'] == 1]['observation_date']

    for rec_qtr in recession_quarters:
        fig.add_shape(
            type="line",
            x0=rec_qtr, x1=rec_qtr,
            y0=0, y1=1,
            xref='x',
            yref='paper',
            line=dict(
                color='rgba(128,128,128,0.6)',  # pink
                width=6,
                dash='solid'  # solid line
            ),
            layer='below'
        )




    fig.update_layout(
        title="Quarterly Estimates of Annual Interest Rates at Various Leverage Ratios for the Years 1996 through 1Q 2025",
        yaxis_title='Estimated Annual Interest Rate Expense',
        legend_title='Legend',
        template='plotly_white',
        hovermode='x unified',
        height=600,
        xaxis=dict(showgrid=False),  # Remove vertical gridlines
        yaxis=dict(showgrid=False),  # Remove horizontal gridlines (primary y-axis)
        yaxis2=dict(showgrid=False),
    )

    fig.update_yaxes(
    title_text='Estimated Annual Interest Rate Expense',
    tickformat='.0%',
    secondary_y=False,
    
    
   
)

    fig.update_yaxes(
        title_text='Quarterly Loan Volume (Commitments) in USD Billions',
        secondary_y=True
    )




    return fig


if __name__ == '__main__':
    import os
    port = int(os.environ.get("PORT", 8050))  # Get port from environment
    app.run(host='0.0.0.0', port=port, debug=True)

ConnectionError: HTTPConnectionPool(host='0.0.0.0', port=8050): Max retries exceeded with url: /_alive_8db4cf16-c107-4606-b28e-faa7189cad52 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000021D516C3BD0>: Failed to establish a new connection: [WinError 10049] The requested address is not valid in its context'))

In [49]:
import pandas as pd
from dash import Dash, dcc, html, Input, Output, State
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Load data
interest_df = pd.read_excel("C:\\Users\\Raabiyaal Ishaq\\OneDrive - The University of Chicago\\Desktop\\2. RA-SHIP\\SUMMER PROJECTS\\ACLI\\Python\\dash-app\\All Prop Type.xlsx", header=1)
recession_df = pd.read_excel("C:\\Users\\Raabiyaal Ishaq\\OneDrive - The University of Chicago\\Desktop\\2. RA-SHIP\\SUMMER PROJECTS\\ACLI\\Python\\dash-app\\Recession Data.xlsx", header=1)
recession_df['observation_date'] = pd.to_datetime(recession_df['observation_date'])
rf_df = pd.read_excel("C:\\Users\\Raabiyaal Ishaq\\OneDrive - The University of Chicago\\Desktop\\2. RA-SHIP\\SUMMER PROJECTS\\ACLI\\Python\\dash-app\\Rf.xlsx", header=1)
rf_df['Quarter'] = pd.to_datetime(rf_df['Quarter'])
rf_df['Rf'] = rf_df['Rf'].astype(str).str.replace('%', '').astype(float)

# Load leading/lagging returns
def clean_percent(x):
    if isinstance(x, str):
        return float(x.replace('%', '')) if '%' in x else float(x)
    return x

returns_df = pd.read_excel("C:\\Users\\Raabiyaal Ishaq\\OneDrive - The University of Chicago\\Desktop\\2. RA-SHIP\\SUMMER PROJECTS\\ACLI\\Python\\dash-app\\NCREIF.xlsx", header=0)
for col in returns_df.columns[:6]:
    returns_df[col] = pd.to_datetime(returns_df[col])
for col in returns_df.columns[6:]:
    returns_df[col] = returns_df[col].apply(clean_percent)

# Process interest data
if 'Unnamed: 0' in interest_df.columns:
    interest_df.drop(columns=['Unnamed: 0'], inplace=True)
interest_df['Quarter'] = pd.to_datetime(interest_df['Quarter'])
for col in ['Rf', 'g', 'LTV: 25%', 'LTV: 50%', 'LTV: 75%']:
    interest_df[col] = interest_df[col].astype(str).str.replace('%', '').astype(float)
interest_df['Loan Committed'] = interest_df['Loan Committed'].astype(str).str.replace(',', '').astype(float).astype(int)

melted = interest_df.melt(
    id_vars=['Property Type', 'Quarter', 'Loan Committed'],
    value_vars=['LTV: 25%', 'LTV: 50%', 'LTV: 75%'],
    var_name='LTV', value_name='Interest Rate')

base_colors = {
    'Industrial': '255, 165, 0',
    'Office': '70, 130, 180',
    'Retail': '255, 105, 180',
    'Apartment': '60, 179, 113',
    'Core': '255, 215, 0'
}

fill_colors = {
    k: {
        f"LTV: {ltv}%": f"rgba({v}, {opacity})" for ltv, opacity in zip([25, 50, 75], [0.5, 0.3, 0.15])
    } for k, v in base_colors.items()
}

def get_line_color(prop): return f"rgba({base_colors.get(prop, '100,100,100')},1)"
def darken_color(rgb, amt=0.5):
    r, g, b = map(int, rgb.split(','))
    return f"rgba({int(r*amt)},{int(g*amt)},{int(b*amt)},1)"
def get_loan_color(prop): return darken_color(base_colors.get(prop, '100,100,100'), 0.5)

leading_lagging_options = [
    "Leading (1year)", "Leading (3years)", "Leading (5years)",
    "Lagging (1year)", "Lagging (3years)", "Lagging (5years)"
]

property_return_column_map = {
    "Apartment": "Apt Total Returns",
    "Core": "Core Total Returns",
    "Industrial": "Ind Total Return",
    "Office": "Off Total Returns",
    "Retail": "Ret Total Returns"
}

app = Dash(__name__)

app.layout = html.Div([
    html.H3("Interactive Dashboard for Real Estate Metrics"),
    dcc.Dropdown(id='property-dropdown', multi=True,
        options=[{'label': pt, 'value': pt} for pt in interest_df['Property Type'].unique()],
        value=['Apartment']),

    html.Label("Select LTVs:"),
    dcc.Checklist(id='ltv-checklist', inline=True,
        options=[{'label': l, 'value': l} for l in ['LTV: 25%', 'LTV: 50%', 'LTV: 75%']],
        value=['LTV: 25%', 'LTV: 50%', 'LTV: 75%']),

    html.Label("Select Add-ons:"),
    dcc.Checklist(id='addon-checklist', inline=True,
        options=[
            {'label': 'Structural Differences', 'value': 'gamma'},
            {'label': 'Risk-Free Rate', 'value': 'rf'},
            {'label': 'Loan Volume', 'value': 'loan'}
        ], value=['gamma', 'rf', 'loan']),

    html.Div(id='return-checklist-container'),

    dcc.Graph(id='main-graph')
])

@app.callback(
    Output('return-checklist-container', 'children'),
    Input('property-dropdown', 'value')
)
def show_return_checklist(selected):
    if not selected:
        return html.Div()
    return html.Div([
        html.Label("Select Leading/Lagging Returns to Overlay:"),
        dcc.Checklist(
            id='return-series-checklist',
            options=[{'label': ll, 'value': ll} for ll in leading_lagging_options],
            value=[],
            inline=True
        )
    ])

@app.callback(
    Output('main-graph', 'figure'),
    Input('property-dropdown', 'value'),
    Input('ltv-checklist', 'value'),
    Input('addon-checklist', 'value'),
    Input('return-series-checklist', 'value')
)
def update_graph(properties, ltvs, addons, return_series):
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    if not properties or not ltvs:
        return fig

    for prop in properties:
        for ltv in ltvs:
            df_sub = melted[(melted['Property Type'] == prop) & (melted['LTV'] == ltv)]
            fig.add_trace(go.Scatter(
                x=df_sub['Quarter'], y=df_sub['Interest Rate'], mode='lines', fill='tonexty',
                name=f"{prop} - {ltv}",
                line=dict(color=get_line_color(prop)),
                fillcolor=fill_colors[prop].get(ltv, 'rgba(200,200,200,0.2)')
            ), secondary_y=False)

        if 'gamma' in addons:
            df_g = interest_df[interest_df['Property Type'] == prop]
            fig.add_trace(go.Scatter(
                x=df_g['Quarter'], y=df_g['g'], name=f"{prop} Structural Diff.",
                line=dict(color=darken_color(base_colors[prop], 0.7), dash='dash')
            ), secondary_y=False)

        if 'loan' in addons:
            df_l = interest_df[interest_df['Property Type'] == prop]
            fig.add_trace(go.Scatter(
                x=df_l['Quarter'], y=df_l['Loan Committed'] / 1e9,
                name=f"{prop} Loan Volume",
                line=dict(color=get_loan_color(prop))
            ), secondary_y=True)

        if return_series:
            for series in return_series:
                x_vals, y_vals = [], []
                for _, row in returns_df.iterrows():
                    x_date = row[series]
                    col_name = property_return_column_map.get(prop)
                    y_val = row.get(col_name, None)
                    if pd.notna(x_date) and pd.notna(y_val):
                        x_vals.append(x_date)
                        y_vals.append(y_val)
                fig.add_trace(go.Scatter(
                    x=x_vals, y=y_vals,
                    mode='lines', name=f"{prop} - {series}",
                    line=dict(dash='dot')
                ), secondary_y=False)

    if 'rf' in addons:
        fig.add_trace(go.Scatter(
            x=rf_df['Quarter'], y=rf_df['Rf'],
            name='Risk-Free Rate', line=dict(color='black', dash='dot')
        ), secondary_y=False)

    for rec_qtr in recession_df[recession_df['USREC'] == 1]['observation_date']:
        fig.add_shape(
            type='line', x0=rec_qtr, x1=rec_qtr, y0=0, y1=1,
            xref='x', yref='paper', line=dict(color='gray', width=5), layer='below')

    fig.update_layout(
        title="Quarterly Real Estate Lending & Return Metrics",
        yaxis_title="Interest Rate / Return", yaxis2_title="Loan Volume (Billion USD)",
        template="plotly_white", height=650, hovermode="x unified",
        yaxis=dict(anchor='x', overlaying='y2', side='left', zeroline=True),
        yaxis2=dict(anchor='x', overlaying='y', side='right', zeroline=True, showgrid=False)
    )
    fig.update_yaxes(tickformat=".0%", secondary_y=False)
    return fig

if __name__ == '__main__':
    app.run(debug=True)


In [63]:
import pandas as pd
from dash import Dash, dcc, html, Input, Output
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Load data
interest_df = pd.read_excel("All Prop Type.xlsx", header=1)
recession_df = pd.read_excel("Recession Data.xlsx", header=1)
recession_df['observation_date'] = pd.to_datetime(recession_df['observation_date'])
rf_df = pd.read_excel("Rf.xlsx", header=1)
rf_df['Quarter'] = pd.to_datetime(rf_df['Quarter'])
rf_df['Rf'] = rf_df['Rf'].astype(str).str.replace('%', '').astype(float)

# Load returns data
def clean_percent(x):
    if isinstance(x, str):
        return float(x.replace('%', '')) if '%' in x else float(x)
    return x

returns_df = pd.read_excel("NCREIF.xlsx", header=0)
for col in returns_df.columns[:7]:  # includes Returns + leading/lagging
    returns_df[col] = pd.to_datetime(returns_df[col])
for col in returns_df.columns[7:]:
    returns_df[col] = returns_df[col].apply(clean_percent)

# Process interest data
if 'Unnamed: 0' in interest_df.columns:
    interest_df.drop(columns=['Unnamed: 0'], inplace=True)
interest_df['Quarter'] = pd.to_datetime(interest_df['Quarter'])
for col in ['Rf', 'g', 'LTV: 25%', 'LTV: 50%', 'LTV: 75%']:
    interest_df[col] = interest_df[col].astype(str).str.replace('%', '').astype(float)
interest_df['Loan Committed'] = interest_df['Loan Committed'].astype(str).str.replace(',', '').astype(float).astype(int)

melted = interest_df.melt(
    id_vars=['Property Type', 'Quarter', 'Loan Committed'],
    value_vars=['LTV: 25%', 'LTV: 50%', 'LTV: 75%'],
    var_name='LTV', value_name='Interest Rate')

# Colors
base_colors = {
    'LTV: 75%': '#3222CE',
    'LTV: 50%': '#7030A0',
    'LTV: 25%': '#00B050'
}

gamma_color = '#C9C9C9'
rf_color = '#A6A6A6'

def darken_color(hex_color, amt=0.5):
    # Convert hex to RGB tuple
    hex_color = hex_color.lstrip('#')
    r, g, b = int(hex_color[0:2],16), int(hex_color[2:4],16), int(hex_color[4:6],16)
    r = int(r * amt)
    g = int(g * amt)
    b = int(b * amt)
    return f'rgb({r},{g},{b})'

leading_lagging_options = [
    {"label": "None", "value": "None"},
    {"label": "Returns", "value": "Returns"},
    {"label": "Leading (1year)", "value": "Leading (1year)"},
    {"label": "Leading (3years)", "value": "Leading (3years)"},
    {"label": "Leading (5years)", "value": "Leading (5years)"},
    {"label": "Lagging (1year)", "value": "Lagging (1year)"},
    {"label": "Lagging (3years)", "value": "Lagging (3years)"},
    {"label": "Lagging (5years)", "value": "Lagging (5years)"}
]

app = Dash(__name__)

app.layout = html.Div([

    html.Label("Select LTVs:"),
    dcc.Checklist(id='ltv-checklist', inline=True,
        options=[{'label': l, 'value': l} for l in ['LTV: 25%', 'LTV: 50%', 'LTV: 75%']],
        value=['LTV: 25%', 'LTV: 50%', 'LTV: 75%']),

    html.Label("Select Add-ons:"),
    dcc.Checklist(id='addon-checklist', inline=True,
        options=[
            {'label': 'Structural Differences', 'value': 'gamma'},
            {'label': 'Risk-Free Rate', 'value': 'rf'},
            {'label': 'Loan Volume', 'value': 'loan'}
        ], value=['gamma', 'rf', 'loan']),

    html.Label("Select NCREIF Return Series to Overlay:"),
    dcc.RadioItems(
        id='return-series-radio',
        options=leading_lagging_options,
        value="None",
        inline=True
    ),

    dcc.Graph(id='main-graph')
])

@app.callback(
    Output('main-graph', 'figure'),
    Input('ltv-checklist', 'value'),
    Input('addon-checklist', 'value'),
    Input('return-series-radio', 'value')
)
def update_graph(ltvs, addons, return_series):
    prop = "Apartment"
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    ordered_ltvs = ['LTV: 75%', 'LTV: 50%', 'LTV: 25%']

    # Plot LTV fills with opaque colors and solid lines
    for ltv in ordered_ltvs:
        if ltv in ltvs:
            df_sub = melted[(melted['Property Type'] == prop) & (melted['LTV'] == ltv)]
            fig.add_trace(go.Scatter(
                x=df_sub['Quarter'], y=df_sub['Interest Rate'], mode='lines',
                name=f"{prop} - {ltv}",
                line=dict(color=base_colors[ltv], width=2),
                fill='tozeroy',
                fillcolor=base_colors[ltv]
            ), secondary_y=False)

    # Structural Differences area and line
    if 'gamma' in addons:
        df_g = interest_df[interest_df['Property Type'] == prop]
        fig.add_trace(go.Scatter(
            x=df_g['Quarter'], y=df_g['g'], name=f"{prop} Structural Diff.",
            mode='lines',
            line=dict(color=gamma_color, width=2),
            fill='tozeroy',
            fillcolor=gamma_color
        ), secondary_y=False)

    # Loan volume on secondary y-axis
    if 'loan' in addons:
        df_l = interest_df[interest_df['Property Type'] == prop]
        fig.add_trace(go.Scatter(
            x=df_l['Quarter'], y=df_l['Loan Committed'] / 1e9,
            name=f"{prop} Loan Volume",
            line=dict(color=darken_color(gamma_color, 0.5), width=2)
        ), secondary_y=True)

    # Risk Free Rate area and line
    if 'rf' in addons:
        fig.add_trace(go.Scatter(
            x=rf_df['Quarter'], y=rf_df['Rf'],
            name='Risk-Free Rate',
            mode='lines',
            line=dict(color=rf_color, width=2),
            fill='tozeroy',
            fillcolor=rf_color
        ), secondary_y=False)

    # Add return/leading/lagging line on top if selected (and not "None")
    if return_series and return_series != "None":
        return_col = "Apt Total Returns"
        x_vals, y_vals = [], []
        for _, row in returns_df.iterrows():
            x_date = row[return_series]
            y_val = row.get(return_col)
            if pd.notna(x_date) and pd.notna(y_val):
                x_vals.append(x_date)
                y_vals.append(y_val)
        if x_vals:
            fig.add_trace(go.Scatter(
                x=x_vals, y=y_vals,
                mode='lines',
                name=f"{prop} - {return_series}",
                line=dict(color='red', width=2),
                fill=None
            ), secondary_y=False)

    # Recession vertical lines
    for rec_qtr in recession_df[recession_df['USREC'] == 1]['observation_date']:
        fig.add_shape(
            type='line', x0=rec_qtr, x1=rec_qtr, y0=0, y1=1,
            xref='x', yref='paper', line=dict(color='gray', width=5), layer='below')

    fig.update_layout(
        # Title removed as requested
        yaxis_title="Interest Rate / Return", yaxis2_title="Loan Volume (Billion USD)",
        template="plotly_white", height=650, hovermode="x unified",
        hoverlabel=dict(
            namelength=50,  # increase max chars shown for trace names
            font=dict(size=13),
            bgcolor="white",
            bordercolor="black"
        )
    )

    fig.update_yaxes(tickformat=".0%", secondary_y=False)

    # Align secondary y-axis 0 with primary y-axis 0
    primary_y_vals = []
    secondary_y_vals = []

    for trace in fig.data:
        if not hasattr(trace, 'y') or trace.y is None:
            continue
        if getattr(trace, 'yaxis', 'y') == 'y2':
            secondary_y_vals.extend([y for y in trace.y if y is not None])
        else:
            primary_y_vals.extend([y for y in trace.y if y is not None])

    if primary_y_vals and secondary_y_vals:
        y1_min, y1_max = min(primary_y_vals), max(primary_y_vals)
        y2_max = max(secondary_y_vals)
        zero_ratio = abs(y1_min) / (y1_max - y1_min) if y1_min < 0 else 0
        y2_min = y2_max * -zero_ratio / (1 - zero_ratio) if zero_ratio < 1 else 0
        fig.update_yaxes(range=[y1_min, y1_max], secondary_y=False)
        fig.update_yaxes(range=[y2_min, y2_max], secondary_y=True)

    return fig

if __name__ == '__main__':
    app.run(debug=True)
